In [27]:
%load_ext autoreload
%autoreload 2

import os
from pprint import pprint

import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.transforms import Affine2D

print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/matthew/git/cadc_testing/uncertainty_eval


In [28]:
dataset_path = '/home/matthew/git/cadc_testing/WISEOpenLidarPerceptron/data/kitti'

# Clustering
logdir = '/home/matthew/git/cadc_testing/pcdet_output/output_pkls'
# preds_path = os.path.join(logdir, 'openpcdet_tracking_result.pkl')
preds_path = os.path.join(logdir, 'mimo_a_tracking2.pkl')
# gts_path = os.path.join(logdir, 'gts.pkl')
gts_path = os.path.join(logdir, 'kitti_infos_val.pkl')


In [29]:
def load_dicts():
    # Load gt and prediction data dict
    with open(gts_path, 'rb') as f:
        gt_dicts = pickle.load(f)
    with open(preds_path, 'rb') as f:
        pred_dicts = pickle.load(f)
    return gt_dicts, pred_dicts

def load_image(frame_id):
    img_path = os.path.join(dataset_path, 'training', 'image_2', f'{frame_id}.png')
    return plt.imread(img_path)

def load_lidar(frame_id, xlim, ylim):
    lidar_path = os.path.join(dataset_path, 'training', 'velodyne', f'{frame_id}.bin')
    points = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 4)
    valid_mask = (points[:,0] > xlim[0]) & \
        (points[:,0] < xlim[1]) & \
        (points[:,1] > ylim[0]) & \
        (points[:,1] < ylim[1]) & \
        (points[:,2] < 4)
    points = points[valid_mask]
    return points

def add_box(ax, box, color=None):
    # box: [x, y, z, w, l, h, theta]
    w = box[3]
    h = box[4]
    xy = (box[0] - w/2, box[1] - h/2)
    angle = box[-1] * 180 / np.pi
    box_rect = Rectangle(
        xy, w, h, facecolor='none', edgecolor=color
    )
    t = Affine2D().rotate_around(box[0], box[1], box[-1]) + ax.transData
    box_rect.set_transform(t)
    ax.add_patch(box_rect)

In [30]:
# Load example GT and prediction
idx = np.random.randint(3700)
# idx = 2322, 291
idx = 1000
print(idx)
gt_dicts, pred_dicts = load_dicts()

print('Example GT dict:')
print(gt_dicts[idx].keys())
print('Example pred dict:')
print(len(pred_dicts))
print(pred_dicts[0])
print(pred_dicts[idx].keys())
print(pred_dicts[idx])
print(pred_dicts[idx]['frame_id'])
print(pred_dicts[idx]['seq_id'])


1000
Example GT dict:
dict_keys(['point_cloud', 'image', 'calib', 'annos'])
Example pred dict:
8004
[{'name': array(['Cyclist', 'Pedestrian', 'Cyclist', 'Car', 'Car', 'Pedestrian',
       'Pedestrian', 'Cyclist', 'Pedestrian', 'Pedestrian', 'Pedestrian',
       'Cyclist', 'Cyclist', 'Car', 'Pedestrian'], dtype='<U10'), 'truncated': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'occluded': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'alpha': array([-8.302435 , -7.697017 , -2.9819405, -7.874996 , -8.095217 ,
       -6.2638903, -4.1227393, -7.1180925, -6.063107 , -2.9738898,
       -5.512026 , -7.772606 , -8.016417 , -7.185817 , -5.1998324],
      dtype=float32), 'bbox': array([[ 742.4585 ,  157.30183,  919.79755,  374.     ],
       [1108.436  ,  166.86644, 1191.941  ,  327.84964],
       [ 820.2415 ,  166.21448,  898.67554,  240.72115],
       [ 779.495  ,  186.99757,  856.8843 ,  265.4444 ],
       [ 298.30914,  160.4555 ,  459.32697,  29

AttributeError: 'list' object has no attribute 'keys'

In [31]:
# Only run for mimo
# Cluster pred_dicts
from cluster import cluster_preds

MIN_CLUSTER_SIZE = 1

print("Before clustering")
print("Number of frames", len(pred_dicts))
print("Number of pred dicts per frame", len(pred_dicts[0]))

pred_dicts = cluster_preds(pred_dicts, MIN_CLUSTER_SIZE, tracking_mode=True)

print("After clustering")
print("Number of frames", len(pred_dicts))
print("Is the output per frame now a single dict?", isinstance(pred_dicts[0], dict))

Before clustering
Number of frames 8004
Number of pred dicts per frame 2
circ mean test
angle in cluster 5.554925
angle in cluster 2.4103785
pred_boxes angle_a 5.554925
pred_boxes angle_b 2.4103785
real mean of angles 0.8410605510049542
fake mean of angles -0.728259952860423
fake mean of angles 2.4103788455301007
circ mean test
angle in cluster 6.192506
angle in cluster 2.6187897
pred_boxes angle_a 6.192506
pred_boxes angle_b 2.6187897
real mean of angles 1.2640555222802838
fake mean of angles -0.09067908127839175
fake mean of angles 2.618790002661296
circ mean test
angle in cluster 6.0580654
angle in cluster 2.9147053
pred_boxes angle_a 6.0580654
pred_boxes angle_b 2.9147053
real mean of angles 1.344787987070628
fake mean of angles -0.22511950333649722
fake mean of angles 2.914705606298991
circ mean test
angle in cluster 6.371235
angle in cluster 3.20885
pred_boxes angle_a 6.371235
pred_boxes angle_b 3.20885
real mean of angles 1.6484494527154645
fake mean of angles 0.0880500396066388

circ mean test
angle in cluster 6.4241667
angle in cluster 3.283794
pred_boxes angle_a 6.4241667
pred_boxes angle_b 3.283794
real mean of angles -1.429208668070384
fake mean of angles 0.14098182519013536
fake mean of angles -2.9993915279679975
circ mean test
angle in cluster 4.3002152
angle in cluster 1.1652583
pred_boxes angle_a 4.3002152
pred_boxes angle_b 1.1652583
real mean of angles 2.7327403147988996
fake mean of angles -1.9829701503091535
fake mean of angles 1.1652582009607038
circ mean test
angle in cluster 4.2175694
angle in cluster 0.8497256
pred_boxes angle_a 4.2175694
pred_boxes angle_b 0.8497256
real mean of angles -0.6079448779397687
fake mean of angles -2.0656156857782086
fake mean of angles 0.8497256358438214
circ mean test
angle in cluster 4.264724
angle in cluster 1.1590451
pred_boxes angle_a 4.264724
pred_boxes angle_b 1.1590451
real mean of angles 2.7118845899873456
fake mean of angles -2.0184612592035016
fake mean of angles 1.1590451319986066
circ mean test
angle i

circ mean test
angle in cluster 6.53051
angle in cluster 3.388348
pred_boxes angle_a 6.53051
pred_boxes angle_b 3.388348
real mean of angles 1.8178241570764264
fake mean of angles 0.24732509453827944
fake mean of angles -2.894837351637431
circ mean test
angle in cluster 1.9021126
angle in cluster 5.053035
pred_boxes angle_a 1.9021126
pred_boxes angle_b 5.053035
real mean of angles 0.33598295052582783
fake mean of angles 1.9021128733926496
fake mean of angles -1.2301502545648297
circ mean test
angle in cluster 6.4132767
angle in cluster 3.2694325
pred_boxes angle_a 6.4132767
pred_boxes angle_b 3.2694325
real mean of angles 1.6997614224725446
fake mean of angles 0.1300918181710915
fake mean of angles -3.013752909498759
circ mean test
angle in cluster 2.9241648
angle in cluster 6.0680437
pred_boxes angle_a 2.9241648
pred_boxes angle_b 6.0680437
real mean of angles 1.3545282204919538
fake mean of angles 2.9241651018434247
fake mean of angles -0.21514120896393907
circ mean test
angle in clu

circ mean test
angle in cluster 4.5707207
angle in cluster 1.431504
pred_boxes angle_a 4.5707207
pred_boxes angle_b 1.431504
real mean of angles 3.0011112709813794
fake mean of angles -1.7124642451577863
fake mean of angles 1.4315041621499738
circ mean test
angle in cluster 3.526013
angle in cluster 6.5218983
pred_boxes angle_a 3.526013
pred_boxes angle_b 6.5218983
real mean of angles -1.2592296918206891
fake mean of angles -2.7571725865178784
fake mean of angles 0.23871341546113145
circ mean test
angle in cluster 5.7934885
angle in cluster 2.668182
pred_boxes angle_a 5.7934885
pred_boxes angle_b 2.668182
real mean of angles -2.0523491223626813
fake mean of angles -0.48969641526276675
fake mean of angles 2.668181749182292
circ mean test
angle in cluster 6.3307943
angle in cluster 3.1983418
pred_boxes angle_a 6.3307943
pred_boxes angle_b 3.1983418
real mean of angles -1.5186133702569684
fake mean of angles 0.04760924180085313
fake mean of angles -3.084843611466237
circ mean test
angle i

circ mean test
angle in cluster 4.732253
angle in cluster 1.435204
pred_boxes angle_a 4.732253
pred_boxes angle_b 1.435204
real mean of angles -0.05786410172516909
fake mean of angles -1.5509319623285016
fake mean of angles 1.4352039416604718
circ mean test
angle in cluster 6.26752
angle in cluster 3.125801
pred_boxes angle_a 6.26752
pred_boxes angle_b 3.125801
real mean of angles 1.5547389109903058
fake mean of angles -0.015664966898508936
fake mean of angles 3.1258014125102243
circ mean test
angle in cluster 4.362911
angle in cluster 1.1773627
pred_boxes angle_a 4.362911
pred_boxes angle_b 1.1773627
real mean of angles -0.37145486672455874
fake mean of angles -1.9202736933999738
fake mean of angles 1.1773628314309796
circ mean test
angle in cluster 4.759975
angle in cluster 1.6179768
pred_boxes angle_a 4.759975
pred_boxes angle_b 1.6179768
real mean of angles 0.04738798935944599
fake mean of angles -1.5232105572992047
fake mean of angles 1.6179769357019147
circ mean test
angle in clu

...

circ mean test
angle in cluster 4.1845655
angle in cluster 1.0768806
pred_boxes angle_a 4.1845655
pred_boxes angle_b 1.0768806
real mean of angles 2.630723925428935
fake mean of angles -2.09861937363679
fake mean of angles 1.0768806060128888
circ mean test
angle in cluster 3.4641876
angle in cluster 0.92794955
pred_boxes angle_a 3.4641876
pred_boxes angle_b 0.92794955
real mean of angles 2.1960687955194196
fake mean of angles -2.8189973552995404
fake mean of angles 0.9279495795541486
circ mean test
angle in cluster 4.017447
angle in cluster 0.8949541
pred_boxes angle_a 4.017447
pred_boxes angle_b 0.8949541
real mean of angles 2.4562006314569196
fake mean of angles -2.2657388766580304
fake mean of angles 0.8949538787179669
circ mean test
angle in cluster 6.734741
angle in cluster 3.7013717
pred_boxes angle_a 6.734741
pred_boxes angle_b 3.7013717
real mean of angles -1.0651284774118146
fake mean of angles 0.4515563567453107
fake mean of angles -2.581813784437724
circ mean test
angle in c

circ mean test
angle in cluster 3.0999327
angle in cluster 6.3010283
pred_boxes angle_a 3.0999327
pred_boxes angle_b 6.3010283
real mean of angles 1.5588879903131208
fake mean of angles 3.0999329929524144
fake mean of angles 0.01784315903718081
circ mean test
angle in cluster 6.3094378
angle in cluster 3.1996737
pred_boxes angle_a 6.3094378
pred_boxes angle_b 3.1996737
real mean of angles -1.5286281426721295
fake mean of angles 0.026252659159251124
fake mean of angles -3.0835113284462174
circ mean test
angle in cluster 6.238742
angle in cluster 3.0889163
pred_boxes angle_a 6.238742
pred_boxes angle_b 3.0889163
real mean of angles 1.522237094240733
fake mean of angles -0.044443043070383936
fake mean of angles 3.0889161509745797
circ mean test
angle in cluster 6.1589003
angle in cluster 2.985805
pred_boxes angle_a 6.1589003
pred_boxes angle_b 2.985805
real mean of angles 1.430760296183177
fake mean of angles -0.12428465683991519
fake mean of angles 2.985804872708864
circ mean test
angle 

circ mean test
angle in cluster 6.261778
angle in cluster 3.122254
pred_boxes angle_a 6.261778
pred_boxes angle_b 3.122254
real mean of angles -1.5911779721551618
fake mean of angles -0.021407039957590968
fake mean of angles 3.122253267216035
circ mean test
angle in cluster 4.541606
angle in cluster 1.3660113
pred_boxes angle_a 4.541606
pred_boxes angle_b 1.3660113
real mean of angles -0.18778410752350894
fake mean of angles -1.7415789683633527
fake mean of angles 1.366011532145091
circ mean test
angle in cluster 6.838991
angle in cluster 3.7396746
pred_boxes angle_a 6.838991
pred_boxes angle_b 3.7396746
real mean of angles -0.9938518126779279
fake mean of angles 0.555806072550364
fake mean of angles -2.543510826426097
circ mean test
angle in cluster 6.853428
angle in cluster 3.7229786
pred_boxes angle_a 6.853428
pred_boxes angle_b 3.7229786
real mean of angles -0.9949821551614484
fake mean of angles 0.5702427943521222
fake mean of angles -2.5602059086137494
circ mean test
angle in clu

circ mean test
angle in cluster 4.450664
angle in cluster 1.3271079
pred_boxes angle_a 4.450664
pred_boxes angle_b 1.3271079
real mean of angles 2.8888871689611157
fake mean of angles -1.8325213511758527
fake mean of angles 1.3271078189187726
circ mean test
angle in cluster 3.4789786
angle in cluster 6.609811
pred_boxes angle_a 3.4789786
pred_boxes angle_b 6.609811
real mean of angles -1.2387895901971542
fake mean of angles -2.8042063434892377
fake mean of angles 0.32662597497040835
circ mean test
angle in cluster 3.0149932
angle in cluster 6.143012
pred_boxes angle_a 3.0149932
pred_boxes angle_b 6.143012
real mean of angles -1.7041793187432965
fake mean of angles 3.014992566900797
fake mean of angles -0.1401728709512433
circ mean test
angle in cluster 4.2440715
angle in cluster 1.1035812
pred_boxes angle_a 4.2440715
pred_boxes angle_b 1.1035812
real mean of angles 2.673833342390605
fake mean of angles -2.0391134341531476
fake mean of angles 1.1035811026864728
circ mean test
angle in c

circ mean test
angle in cluster 2.0649095
angle in cluster 5.209939
pred_boxes angle_a 2.0649095
pred_boxes angle_b 5.209939
real mean of angles 0.49583164055878726
fake mean of angles 2.0649092515283307
fake mean of angles -1.0732461531930646
circ mean test
angle in cluster 3.425726
angle in cluster 6.292368
pred_boxes angle_a 3.425726
pred_boxes angle_b 6.292368
real mean of angles -1.4241381009393412
fake mean of angles -2.8574590404802045
fake mean of angles 0.009182842569895655
circ mean test
angle in cluster 6.641778
angle in cluster 3.4593725
pred_boxes angle_a 6.641778
pred_boxes angle_b 3.4593725
real mean of angles 1.908983739214488
fake mean of angles 0.35859313805634585
fake mean of angles -2.823812486725398
circ mean test
angle in cluster 2.370728
angle in cluster 5.5253944
pred_boxes angle_a 2.370728
pred_boxes angle_b 5.5253944
real mean of angles 0.806469591456004
fake mean of angles 2.3707278688722333
fake mean of angles -0.7577904780679425
circ mean test
angle in clus

In [32]:
print(pred_dicts[0])

{'frame_id': '000000', 'seq_id': '0000', 'name': array(['Cyclist', 'Pedestrian', 'Cyclist', 'Car', 'Car', 'Pedestrian',
       'Pedestrian', 'Cyclist', 'Pedestrian', 'Pedestrian', 'Pedestrian',
       'Cyclist', 'Cyclist', 'Car', 'Pedestrian', 'Pedestrian', 'Cyclist',
       'Pedestrian'], dtype='<U10'), 'pred_labels': array([3, 2, 3, 1, 1, 2, 2, 3, 2, 2, 2, 3, 3, 1, 2, 2, 3, 2]), 'score': array([0.78885174, 0.6883113 , 0.3428439 , 0.28993917, 0.280379  ,
       0.23641092, 0.16861309, 0.16267794, 0.16017921, 0.16874957,
       0.12139743, 0.1141316 , 0.10920072, 0.1072453 , 0.10137714,
       0.20789458, 0.11300042, 0.10916225], dtype=float32), 'score_all': array([[1.43394949e-08, 1.43493377e-08, 7.88851678e-01, 2.11148277e-01],
       [1.37662965e-08, 6.88311338e-01, 1.37845975e-08, 3.11688721e-01],
       [7.04640968e-08, 7.03856102e-08, 3.42843831e-01, 6.57155991e-01],
       [2.89939165e-01, 7.53805551e-09, 7.54329577e-09, 7.10060716e-01],
       [2.80378997e-01, 2.78454904e-09, 2

In [34]:
# Create directories

trackinglogdir = '/home/matthew/git/cadc_testing/pcdet_output/tracking_output'
model_name = 'openpcdet'
model_name = 'mimo'

carlogdir = trackinglogdir + '/' + model_name + '_Car_val'
pedlogdir = trackinglogdir + '/' + model_name + '_Pedestrian_val'
cyclogdir = trackinglogdir + '/' + model_name + '_Cyclist_val'

os.mkdir(trackinglogdir)
os.mkdir(carlogdir)
os.mkdir(pedlogdir)
os.mkdir(cyclogdir)

In [35]:
name_to_label = { 'Pedestrian': 1, 'Car':2, 'Cyclist': 3 }
curr_seq = None
car_output_file = None
ped_output_file = None
cyc_output_file = None

OUTPUT_VAR = True

for pred_dict in pred_dicts:
    if curr_seq != pred_dict['seq_id']:
        curr_seq = pred_dict['seq_id']
        print('New seq found', curr_seq)
        car_output_file = open(carlogdir + "/" + curr_seq +  ".txt", "a")
        ped_output_file = open(pedlogdir + "/" + curr_seq +  ".txt", "a")
        cyc_output_file = open(cyclogdir + "/" + curr_seq +  ".txt", "a")

    # Loop through objects in frame
    for i in range(len(pred_dict['name'])):
        # The output AB3DMOT requries
        label = name_to_label[pred_dict['name'][i]]
        output = [int(pred_dict['frame_id']),
                  label,
                  pred_dict['bbox'][i][0], pred_dict['bbox'][i][1], pred_dict['bbox'][i][2], pred_dict['bbox'][i][3], #x1,y1,x2,y2
                  pred_dict['score'][i],
                  pred_dict['dimensions'][i][2], pred_dict['dimensions'][i][1], pred_dict['dimensions'][i][0], # h,w,l
                  pred_dict['location'][i][0], pred_dict['location'][i][1], pred_dict['location'][i][2], # x,y,z
                  pred_dict['rotation_y'][i], # rot_y
                  pred_dict['alpha'][i]
                  ]
        if OUTPUT_VAR == True:
            # h,w,l
            output.append(pred_dict['pred_vars'][i][5])
            output.append(pred_dict['pred_vars'][i][4])
            output.append(pred_dict['pred_vars'][i][3])
            # x,y,z
            output.append(pred_dict['pred_vars'][i][0]) # TODO: These are wrong, not in camera frame
            output.append(pred_dict['pred_vars'][i][1])
            output.append(pred_dict['pred_vars'][i][2])
            # rotation_y
            output.append(pred_dict['pred_vars'][i][6])

        if label == 1:
            print(",".join([str(j) for j in output]), file=ped_output_file)
        elif label == 2:
            print(",".join([str(j) for j in output]), file=car_output_file)
        elif label == 3:
            print(",".join([str(j) for j in output]), file=cyc_output_file)
        else:
            raise NotImplemented

New seq found 0000
New seq found 0001
New seq found 0002
New seq found 0003
New seq found 0004
New seq found 0005
New seq found 0006
New seq found 0007
New seq found 0008
New seq found 0009
New seq found 0010
New seq found 0011
New seq found 0012
New seq found 0013
New seq found 0014
New seq found 0015
New seq found 0016
New seq found 0017
New seq found 0018
New seq found 0019
New seq found 0020
